<a href="https://colab.research.google.com/github/shunrei9841-sudo/Guadalupe/blob/main/Parcial%202-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Parámetros del modelo
mu = 0.002
sigma = 0.01
P0 = 10.0
K = 11.0
T = 50.0

# Configuración de simulación
N = 2_000_000  # número de simulaciones (ajusta según tu equipo)
rng = np.random.default_rng(42)  # semilla reproducible

# --- A) Estimación Monte Carlo del valor esperado de la ganancia ---
# Simulación directa de P_T usando la forma cerrada del GBM
# W_T ~ N(0, T) => podemos muestrear Z ~ N(0,1) y usar W_T = sqrt(T) * Z
Z = rng.standard_normal(N)
PT = P0 * np.exp((mu - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

payoff = np.maximum(PT - K, 0.0)
est_plain = payoff.mean()
var_plain = payoff.var(ddof=1)

print(f"Estimación Monte Carlo (simple) del valor esperado de la ganancia: {est_plain:.6f}")
print(f"Varianza de la estimación (simple): {var_plain:.6f}")

# --- B) Reducción de varianza con variables antitéticas ---
# Construimos pares antitéticos (Z, -Z). Esto correlaciona negativamente las muestras.
Z_plus = Z
Z_minus = -Z

PT_plus = P0 * np.exp((mu - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z_plus)
PT_minus = P0 * np.exp((mu - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z_minus)

payoff_plus = np.maximum(PT_plus - K, 0.0)
payoff_minus = np.maximum(PT_minus - K, 0.0)

# Estimador antitético: promedio de cada par
payoff_pair_avg = 0.5 * (payoff_plus + payoff_minus)

est_anti = payoff_pair_avg.mean()
var_anti = payoff_pair_avg.var(ddof=1)

# Porcentaje de reducción de varianza
var_reduction_pct = 100.0 * (var_plain - var_anti) / var_plain

print(f"Estimación Monte Carlo (antitética) del valor esperado de la ganancia: {est_anti:.6f}")
print(f"Varianza del estimador (antitético): {var_anti:.6f}")
print(f"Reducción de varianza por antitética: {var_reduction_pct:.2f}%")

Estimación Monte Carlo (simple) del valor esperado de la ganancia: 0.337664
Varianza de la estimación (simple): 0.243234
Estimación Monte Carlo (antitética) del valor esperado de la ganancia: 0.337428
Varianza del estimador (antitético): 0.064573
Reducción de varianza por antitética: 73.45%
